ImageDataGenerator
- 성능을 올리면서 과적합 방지
- 파이프라인
- 전처리
- 이미지의 다양성
- https://techblog-history-younghunjo1.tistory.com/252

- 정규화 - 특징을 더 잘 추출하기 위해
- 범주화 - 이미지를 더 잘 분류하기 위해

In [ ]:
# 디폴트 batch_size = 32
# x_train / 32
# 90번
# 이미지 장수 = step_per_epoch = 2 * batch_size

In [ ]:
# # 재실행을 위해 필요한 라이브러리를 다시 가져옵니다.
# import os

# # 원본 데이터셋 경로
# source_dataset_path = '/content/drive/MyDrive/딥러닝프로젝트_쓰레기분류모델_CNN/data/current_dataset'

# # 'trash' 카테고리에 해당하는 경로 설정
# trash_category_path = os.path.join(source_dataset_path, 'trash')

# # 'trash' 카테고리 폴더 내의 이미지 파일 수를 확인합니다.
# if os.path.exists(trash_category_path):
#     trash_image_files = [f for f in os.listdir(trash_category_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
#     trash_image_count = len(trash_image_files)
# else:
#     trash_image_count = 0

# trash_image_count

# 세팅

In [ ]:
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
import numpy as np
import shutil

# 원본 데이터셋 경로
source_dataset_path = '/content/drive/MyDrive/딥러닝프로젝트_쓰레기분류모델_CNN/data/current_dataset'
resized_dataset_path = '/content/drive/MyDrive/딥러닝프로젝트_쓰레기분류모델_CNN/data/resized_dataset'
model_path = '/content/drive/MyDrive/딥러닝프로젝트_쓰레기분류모델_CNN/data/model'


# 쓰레기 카테고리
categories = ['metal', 'paper', 'plastic', 'trash', 'cardboard', 'glass']
sizes = []

# 테스트를 위해 갯수 제한
images_per_category = 50

# 이미지 리사이즈 크기 결정
resize_width, resize_height = 384, 512

# 리사이즈 이미지 설정

In [ ]:
for category in categories:
    category_path = os.path.join(source_dataset_path, category)
    # 카테고리 폴더 내의 이미지 중 png,jpg,jpeg 필터링
    # 앞의 f 필터링된 파일이름
    # 뒤의 f 카테고리 폴더의 모든 파일
    # f.lower().endswith(('.png', '.jpg', '.jpeg') -> 파일이름을 소문자로 변경하고
    # 해당 이름이 '.png', '.jpg', 또는 '.jpeg'로 끝나는지를 검사
    image_files = [f for f in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # 각 카테고리별로 지정된 수의 이미지만 처리
    for filename in image_files[:images_per_category]:
        file_path = os.path.join(category_path, filename)
        # OpenCV를 사용하여 이미지를 로드하고, 이미지의 크기를 sizes 리스트에 추가합니다.
        image = cv2.imread(file_path)
        if image is not None:
            sizes.append(image.shape[:2])  # 이미지의 높이와 너비만 추출

In [ ]:
sizes

In [ ]:
# 이미지 크기 통계 계산
if sizes:
    heights, widths = zip(*sizes)
    avg_height = sum(heights) / len(heights)
    avg_width = sum(widths) / len(widths)
    min_height = min(heights)
    min_width = min(widths)
    median_height = sorted(heights)[len(heights) // 2]
    median_width = sorted(widths)[len(widths) // 2]
else:
    avg_height = avg_width = min_height = min_width = median_height = median_width = 0

(avg_height, avg_width), (min_height, min_width), (median_height, median_width)

((384.0, 512.0), (384, 512), (384, 512))

# 이미지 리사이즈

In [ ]:
# 카테고리별로 폴더를 순회하며 이미지 처리
for category in categories:
    source_category_path = os.path.join(source_dataset_path, category)
    resized_category_path = os.path.join(resized_dataset_path, category)
    os.makedirs(resized_category_path, exist_ok=True)  # 리사이즈된 이미지 저장 폴더 생성
    # exist_ok=True 해당 디렉토리에 폴더가 존재해도 오류를 발생시키지 않고 넘어감

    # 이미지 파일 처리
    image_files = os.listdir(source_category_path)[:images_per_category]  # 각 폴더별로 처음 50개의 파일만 가져옴
    for filename in image_files:
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(source_category_path, filename)
            image = cv2.imread(file_path)
            if image is not None:
                resized_image = cv2.resize(image, (resize_width, resize_height))
                cv2.imwrite(os.path.join(resized_category_path, filename), resized_image)

# 데이터 전처리 및 분할

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,  # 회전 범위를 줄임
    width_shift_range=0.1,  # 이동 범위를 줄임
    height_shift_range=0.1,  # 이동 범위를 줄임
    shear_range=0.1,  # 전단 변환 범위를 줄임
    zoom_range=0.1,  # 확대/축소 범위를 줄임
    horizontal_flip=True, # 무작위 수평 뒤집기
    fill_mode='nearest', # 회전 또는 너비/높이 이동으로 인해 새롭게 생성해야 할 픽셀을 채우는 방식
    validation_split=0.2  # 데이터셋의 20%를 검증 데이터로 사용
)

# 참고 : https://tykimos.github.io/2017/06/10/CNN_Data_Augmentation/

# 학습 데이터셋 로더 설정
train_generator = train_datagen.flow_from_directory(
    resized_dataset_path,
    target_size=(resize_width, resize_height),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # 학습 데이터셋
)

# 검증 데이터셋 로더 설정
validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # 검증 데이터셋도 정규화 필요
validation_generator = validation_datagen.flow_from_directory(
    resized_dataset_path,
    target_size=(resize_width, resize_height),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # 검증 데이터셋
)

Found 240 images belonging to 6 classes.
Found 60 images belonging to 6 classes.


In [ ]:
len(train_generator)

8

In [ ]:
len(validation_generator)

2

# 모델 정의 (Define Network)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 모델 구축
model = Sequential([
    # 첫 번째 컨볼루션 레이어
    Conv2D(32, (3, 3), activation='relu', input_shape=(resize_width, resize_height, 3)),
    MaxPooling2D(2, 2),
    # 두 번째 컨볼루션 레이어
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    # 세 번째 컨볼루션 레이어
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    # 네 번째 컨볼루션 레이어
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    # Flatten 레이어로 차원 축소
    Flatten(),
    # 완전 연결 레이어
    Dense(128, activation='relu'),  # 예: 128 뉴런으로 변경
    # 출력 레이어
    Dense(6, activation='softmax')  # 클래스가 6개이므로 뉴런 수를 6개로 설정
])

# 모델 요약
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 382, 510, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 191, 255, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 189, 253, 64)      18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 94, 126, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 92, 124, 128)      73856     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 46, 62, 128)      

# 모델 컴파일 (Compile Network)
- keras코드 -> tensorflow 코드로 변환

In [ ]:
from tensorflow.keras.optimizers import Adam

# 모델 컴파일
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 모델 학습 (Fit Network)

In [ ]:
# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),  # 학습 데이터셋의 이미지 수를 배치 크기로 나눈 값
    epochs=20,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)  # 검증 데이터셋의 이미지 수를 배치 크기로 나눈 값
)
# 수업 때 배운 걸로 모델 저장하면서 학습
model_save_path = os.path.join(model_path, 'my_model.h5')
model.save(model_save_path)

Epoch 1/20
8/8 [==============================] - 12s 1s/step - loss: 2.1000 - accuracy: 0.1542 - val_loss: 1.7904 - val_accuracy: 0.1833
Epoch 2/20
8/8 [==============================] - 10s 1s/step - loss: 1.7687 - accuracy: 0.2625 - val_loss: 1.7033 - val_accuracy: 0.3000
Epoch 3/20
8/8 [==============================] - 10s 1s/step - loss: 1.7159 - accuracy: 0.2500 - val_loss: 1.7232 - val_accuracy: 0.2500
Epoch 4/20
8/8 [==============================] - 11s 1s/step - loss: 1.7003 - accuracy: 0.2875 - val_loss: 1.6670 - val_accuracy: 0.3167
Epoch 5/20
8/8 [==============================] - 10s 1s/step - loss: 1.6416 - accuracy: 0.2708 - val_loss: 1.6459 - val_accuracy: 0.3167
Epoch 6/20
8/8 [==============================] - 10s 1s/step - loss: 1.5357 - accuracy: 0.3833 - val_loss: 1.5975 - val_accuracy: 0.4000
Epoch 7/20
8/8 [==============================] - 10s 1s/step - loss: 1.4849 - accuracy: 0.3708 - val_loss: 1.4300 - val_accuracy: 0.3500
Epoch 8/20
8/8 [==================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# 모델 평가 (Evaluate Network)

In [ ]:
# 모델 평가
validation_loss, validation_accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {validation_loss}")
print(f"Validation Accuracy: {validation_accuracy}")

2/2 [==============================] - 1s 223ms/step - loss: 1.2479 - accuracy: 0.5333
Validation Loss: 1.2478619813919067
Validation Accuracy: 0.5333333611488342


# 예측 수행 (Make Predictions)
: 쓰레기 이미지를 넣어 쓰레기의 종류를 분류

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# 이미지를 불러오고 전처리하는 함수
def load_and_preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(resize_width, resize_height))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # 모델의 예상 입력 형태에 맞게 차원 추가
    img_array /= 255.0  # 이미지 정규화
    return img_array

# 테스트 이미지 폴더 경로
test_images_path = '/content/drive/MyDrive/딥러닝프로젝트_쓰레기분류모델_CNN/data/test'

# 테스트 이미지 파일 목록 가져오기
test_image_files = [os.path.join(test_images_path, f) for f in os.listdir(test_images_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# 각 이미지에 대해 예측 수행
for image_file in test_image_files:
    img_array = load_and_preprocess_image(image_file)
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class_name = class_names[predicted_class_index]
    print(f"Image: {image_file}, Predicted class: {predicted_class_name}")


1/1 [==============================] - 0s 19ms/step
Image: /content/drive/MyDrive/딥러닝프로젝트_쓰레기분류모델_CNN/data/test/metal72.jpg, Predicted class: plastic
1/1 [==============================] - 0s 20ms/step
Image: /content/drive/MyDrive/딥러닝프로젝트_쓰레기분류모델_CNN/data/test/metal70.jpg, Predicted class: glass
1/1 [==============================] - 0s 20ms/step
Image: /content/drive/MyDrive/딥러닝프로젝트_쓰레기분류모델_CNN/data/test/metal71.jpg, Predicted class: plastic
1/1 [==============================] - 0s 21ms/step
Image: /content/drive/MyDrive/딥러닝프로젝트_쓰레기분류모델_CNN/data/test/cardboard38.jpg, Predicted class: cardboard
1/1 [==============================] - 0s 20ms/step
Image: /content/drive/MyDrive/딥러닝프로젝트_쓰레기분류모델_CNN/data/test/cardboard36.jpg, Predicted class: metal
1/1 [==============================] - 0s 19ms/step
Image: /content/drive/MyDrive/딥러닝프로젝트_쓰레기분류모델_CNN/data/test/cardboard37.jpg, Predicted class: plastic
1/1 [==============================] - 0s 20ms/step
Image: /content/drive/MyDrive/딥러닝프로젝트_